# Case Study - Recommender EDA

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import scipy.stats as scs
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from bs4 import BeautifulSoup as bs

In [ ]:
# Target output
target = pd.read_csv('data/sample_submission.csv')
target.head()

In [ ]:
ratings_df = pd.read_table('data/ratings.dat')
ratings_df.head()

In [13]:
jokes = []
joke = ''
with open('data/jokes.txt') as f:

    for line in f:
        line_list = line.split(' ')
        if line_list[0][-1:] == ':':
            jokes.append(joke)    # add the last joke into the jokes list as a new joke has begun 
            joke = ' '.join(line_list[1:])
        else:
            joke += line
jokes.append(joke)
jokes = jokes[1:]
print(jokes[5])    

Bill and Hillary Clinton are on a trip back to Arkansas. They're almost out of gas, so Bill pulls into a service station on the outskirts of town. The attendant runs out of the station to serve them when Hillary realizes it's an old boyfriend from high school. She and the attendant chat as he gases up their car and cleans the windows. Then they all say goodbye.

As Bill pulls the car onto the road, he turns to Hillary and says, "Now aren't you glad you married me and not him? You could've been the wife of a grease monkey!"

To which Hillary replied, "No, Bill. If I would have married him, you'd be pumping gas and he would be the President!" 



In [ ]:
# filename = 'data/jokes.dat'
# html = open(filename,'r').read()
# data = bs(html, 'html.parser')

# jokes = [joke.text.replace('\n', '').replace('\r', '') for joke in data.select('p')]

In [ ]:
jokes[0]

In [17]:
import pandas as pd
import graphlab as gl
from bs4 import BeautifulSoup


def add_joke_len(filename):
    r = open(filename)
    soup = BeautifulSoup(r, 'html.parser')

    joke_list = soup.find_all('p')

    joke_len_list = []
    breaks_list = []
    q_list = []

    for x in joke_list:
        q = 0
        breaks = len(x.find_all('br'))
        breaks_list.append(breaks)
        joke_len = 0
        for y in x:
            joke_len += len(y)
            if y.find('?') > 0:
                q = 1
        joke_len_list.append(joke_len)
        q_list.append(q)

    return q_list, breaks_list, joke_len_list



sample_sub_fname = "data/sample_submission.csv"
ratings_data_fname = "data/ratings.dat"
output_fname = "data/test_ratings.csv"

filename = 'data/jokes.dat'
q_list, breaks_list, joke_len = add_joke_len(filename)

joke_info = gl.SFrame({'joke_id': range(1,151), 'joke_len': joke_len, 'joke_break': breaks_list, 'question': q_list})

ratings = gl.SFrame(ratings_data_fname, format='tsv')
sample_sub = pd.read_csv(sample_sub_fname)
for_prediction = gl.SFrame(sample_sub)
rec_engine = gl.factorization_recommender.create(observation_data=ratings,
                                                 user_id="user_id",
                                                 item_id="joke_id",
                                                 target='rating',
                                                 item_data = joke_info,
                                                 solver='auto')

sample_sub.rating = rec_engine.predict(for_prediction)
sample_sub.to_csv(output_fname, index=False)


Finished parsing file /Users/souljourner/Library/Mobile Documents/com~apple~CloudDocs/Development/Data Science/zipfian/case-study-recommender/data/ratings.dat

Parsing completed. Parsed 100 lines in 0.364178 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/souljourner/Library/Mobile Documents/com~apple~CloudDocs/Development/Data Science/zipfian/case-study-recommender/data/ratings.dat

Parsing completed. Parsed 1218325 lines in 0.577221 secs.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 10.6

Final training RMSE: 3.25568

In [15]:
gl.factorization_recommender.create?

In [21]:
import pandas as pd
import numpy as np
import graphlab as gl
from bs4 import BeautifulSoup
from pprint import pprint


def add_joke_len(filename):
    r = open(filename)
    soup = BeautifulSoup(r, 'html.parser')

    joke_list = soup.find_all('p')

    joke_len_list = []
    q_list = []
    clinton_list = []

    for x in joke_list:
        if x.find('?') > 0:
            q = 1
        else:
            q = 0
        if x.find('Clinton') > 0:
            clinton = 1
        else:
            clinton = 0
        joke_len_list.append(np.log(len(x)))
        q_list.append(q)
        clinton_list.append(clinton)

    return q_list, joke_len_list, clinton_list


sample_sub_fname = "data/sample_submission.csv"
ratings_data_fname = "data/ratings.dat"
output_fname = "data/test_ratings.csv"
test_set = "data/validation_data.csv"

filename = 'data/jokes.dat'
q_list, joke_len, clinton_list = add_joke_len(filename)

#joke_info = gl.SFrame({'joke_id': range(1,151), 'joke_len': joke_len, 'question': q_list})
joke_info = gl.SFrame({'joke_id': range(1,151),  'question': q_list, 'clinton': clinton_list})

ratings = gl.SFrame(ratings_data_fname, format='tsv')
sample_sub = pd.read_csv(sample_sub_fname)
for_prediction = gl.SFrame(sample_sub)
rec_engine = gl.factorization_recommender.create(observation_data=ratings,
                                                 user_id="user_id",
                                                 item_id="joke_id",
                                                 target='rating',
                                                 item_data = joke_info,
                                                 solver='auto',
                                                 regularization=0)

sample_sub.rating = rec_engine.predict(for_prediction)
sample_sub.rating = [min(max(x, -10),10) for x in sample_sub.rating]
sample_sub.to_csv(output_fname, index=False)

print("cross validation")
kfolds = gl.cross_validation.KFold(ratings, 5)
params = dict(user_id='user_id', item_id='joke_id', target='rating',
                  solver='auto', side_data_factorization=False)

print("parameter search")
paramsearch = gl.model_parameter_search.create(
                    kfolds,
                    gl.recommender.factorization_recommender.create,
                    params)

print "best params by rmse:"
pprint(paramsearch.get_best_params('mean_validation_rmse'))


Finished parsing file /Users/souljourner/Library/Mobile Documents/com~apple~CloudDocs/Development/Data Science/zipfian/case-study-recommender/data/ratings.dat

Parsing completed. Parsed 100 lines in 0.370777 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/souljourner/Library/Mobile Documents/com~apple~CloudDocs/Development/Data Science/zipfian/case-study-recommender/data/ratings.dat

Parsing completed. Parsed 1218325 lines in 0.571563 secs.

...

Final training RMSE: 3.00674

[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.job: Creating a LocalAsync environment called 'async'.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Mar-07-2017-14-34-4600000' ready for execution


cross validation
parameter search


[INFO] graphlab.deploy.map_job: Job: 'Model-Parameter-Search-Mar-07-2017-14-34-4600000' scheduled.
[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: A job with name 'Model-Parameter-Search-Mar-07-2017-14-34-4600000' already exists. Renaming the job to 'Model-Parameter-Search-Mar-07-2017-14-34-4600000-92fac'.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Mar-07-2017-14-34-4600000-92fac' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Model-Parameter-Search-Mar-07-2017-14-34-4600000-92fac' scheduled.
[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Mar-07-2017-14-34-4600001' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Model-Parameter-Search-Mar-07-2017-14-34-4600001' scheduled.
[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Mar-07-2017-14-34-4600002'

best params by rmse:
{'item_id': 'joke_id',
 'linear_regularization': 1e-09,
 'max_iterations': 50,
 'num_factors': 64,
 'regularization': 1e-06,
 'side_data_factorization': False,
 'solver': 'auto',
 'target': 'rating',
 'user_id': 'user_id'}


In [22]:
print('hi')

hi


In [28]:
rec_engine = gl.factorization_recommender.create(item_data=joke_info, item_id='joke_id', linear_regularization=1e-09,max_iterations=50,num_factors=64,regularization=1e-06,side_data_factorization=False, solver='auto',target='rating',user_id ='user_id')

TypeError: create() takes at least 1 argument (10 given)

In [33]:
rec_engine = gl.factorization_recommender.create(params, item_data = joke_info)

Recsys training: model = factorization_recommender

RuntimeError: Runtime Exception. boost::bad_get: failed value get using boost::get